<a href="https://colab.research.google.com/github/vinos-app/cdn-json/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22pandas_sql_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [ ]:
# В таблице user_actions сохраняются действия покупателей в интернет магазине, структура таблицы

# user_id    - id пользователя
# product_id - id товара
# action     - действие, просмотр товара, сохранение в корзину, покупка
# time       - время совершения события
# date       - дата события

# Для каждого дня рассчитайте, какой процент просмотров товаров завершился покупкой

In [ ]:
all_user_ids = np.arange(1, 1001)
all_product_ids = np.arange(1, 101)

In [ ]:
n = 10000

In [ ]:
user_ids    = np.random.choice(all_user_ids, n)
product_ids = np.random.choice(all_product_ids, n)

In [ ]:
start_date = pd.to_datetime('2022-01-01')

In [ ]:
times = pd.date_range(start_date, periods=n, freq='1min')

In [ ]:
user_actions = pd.DataFrame({'user_id': user_ids, 
                             'product_id': product_ids, 
                             'time': times})

In [ ]:
user_actions['action'] = 'view'

In [ ]:
user_actions.head()

,user_id,product_id,time,action
0,833,74,2022-01-01 00:00:00,view
1,32,48,2022-01-01 00:01:00,view
2,559,91,2022-01-01 00:02:00,view
3,39,36,2022-01-01 00:03:00,view
4,554,53,2022-01-01 00:04:00,view


In [ ]:
def generate_funel_actions(user_id, product_id, time):
    to_cart = 0.2
    to_purchase = 0.4
    
    df = pd.DataFrame()
    
    if np.random.binomial(1, to_cart, 1)[0]:
        df = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(5, unit='s'), 
                           'action'    : 'add to cart'}, index=[0])
        
        if np.random.binomial(1, to_purchase, 1)[0]:
            df_purchase = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(10, unit='s'), 
                           'action'    : 'purchase'}, index=[0])
    
            df = df.append(df_purchase)
    return df

In [ ]:
to_cart_df = pd.DataFrame()

In [ ]:
for index, row in user_actions.iterrows():
    user_df = generate_funel_actions(row['user_id'], row['product_id'], row['time'])
    to_cart_df = to_cart_df.append(user_df)
    

In [ ]:
user_actions = user_actions.append(to_cart_df)
user_actions = user_actions.sort_values('time')

In [ ]:
user_actions['date'] = user_actions.time.dt.date

In [ ]:
user_actions.head()

,user_id,product_id,time,action,date
0,833,74,2022-01-01 00:00:00,view,2022-01-01
1,32,48,2022-01-01 00:01:00,view,2022-01-01
2,559,91,2022-01-01 00:02:00,view,2022-01-01
0,559,91,2022-01-01 00:02:05,add to cart,2022-01-01
3,39,36,2022-01-01 00:03:00,view,2022-01-01


In [ ]:
q = """SELECT date, 
              views, 
              carts, 
              purchases, 
              100 * purchases / views as purchase_percantage 
       FROM (
           SELECT date, 
            count(case when action = 'view'        then 1 else NULL end) as views, 
            count(case when action = 'add to cart' then 1 else NULL end) as carts, 
            count(case when action = 'purchase'    then 1 else NULL end) as purchases 
           FROM user_actions 
           GROUP BY date);"""


In [ ]:
sqldf(q)

,date,views,carts,purchases,purchase_percantage
0,2022-01-01,1440,312,117,8
1,2022-01-02,1440,263,107,7
2,2022-01-03,1440,274,95,6
3,2022-01-04,1440,284,112,7
4,2022-01-05,1440,273,101,7
5,2022-01-06,1440,303,116,8
6,2022-01-07,1360,298,118,8
